### Summary of Project, NLP_with_Wikipedia

The objective of this assignment is to engineer a novel wikipedia search engine using what you've learned about data collection, infrastructure, and natural language processing.

The project has two components:
- Data collection
- Search algorithm development

### Summary of Notebook:

The following is notebook 3 of 3 for this project, NLP_with_Wikipedia.

Steps contained in this notebook specifically:
1. Read in and apply `TfidfVectorizer` to dataset
2. Create `search_terms` and apply `TfidfVectorizer` to those terms
3. Use `cosine_similarity` to find top 5 articles most related to a single term from `search_terms`

----------------------

### Initialization

In [1]:
% run __init__.py
% run nlp.py
% matplotlib inline

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read in data

In [2]:
cd ..

/home/jovyan/Documents/GA_DSI/Projects/project_4


In [3]:
wiki_df = pd.read_pickle('data/wiki_df.p')

### Tfidf Vectorizing and Term Matrix DF Construction

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [6]:
wiki_df['class_label'] = le.fit_transform(wiki_df['parent_category'])

In [7]:
wiki_df['class_label'].unique()

array([1, 0])

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,1), min_df = 1, stop_words = 'english')

In [10]:
term_matrix = tfidf.fit_transform(wiki_df['text_clean'])

In [11]:
term_matrix_df = pd.DataFrame(term_matrix.toarray(), 
                             index=wiki_df.text_clean,
                             columns=tfidf.get_feature_names())

### Create `search_terms` to use for testing search capabilities

In [14]:
search_terms = [
    {'term':'software'}, 
    {'term':'language'}, 
    {'term':'system'}, 
    {'term':'mysql'}
]

In [15]:
search_terms_df = pd.DataFrame(search_terms)

### TFIDF_Vectorize `search_terms`

In [17]:
search_terms_matrix = tfidf.transform(search_terms_df.term)

In [18]:
search_terms_matrix_df = pd.DataFrame(search_terms_matrix.toarray(), 
                                     index=search_terms_df.term, 
                                     columns=tfidf.get_feature_names())

### Select a `search_term` and run a search

In [20]:
np.random.seed(22)

In [21]:
random_search_term_df = search_terms_matrix_df.sample()
random_search_term_df

,aa,aaa,aaaa,aaaaj,aaabc,aaabcaaabcaaabcaaabc,aaai,aaajxb,aaas,aab,...,zylab,zynga,zynx,zynxambulatory,zynxcare,zynxevidence,zynxorder,zyrion,zz,zzap
term,,,,,,,,,,,,,,,,,,,,,
mysql,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df_search = term_matrix_df.append(random_search_term_df)

In [23]:
n_components = 50
SVD = TruncatedSVD(n_components)

In [24]:
component_names = ["component_"+str(i+1) for i in range(n_components)]

In [25]:
SVD_data = SVD.fit_transform(df_search)

In [26]:
SVD_df = pd.DataFrame(SVD_data, 
                     index=df_search.index, 
                     columns=component_names)

### Cosine Similarity to find top 5 related articles to `search_term`

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
SVD_term_vect = SVD_df.loc[random_search_term_df.index]
SVD_term_vect

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_41,component_42,component_43,component_44,component_45,component_46,component_47,component_48,component_49,component_50
mysql,0.008956,-0.007938,-0.006568,-0.007648,-0.020818,0.004924,0.009098,-0.018215,0.010778,0.004295,...,-0.005058,-0.01955,-0.005789,-0.016604,-0.005257,-0.003277,-0.017035,-0.012724,-0.016574,-0.000082


In [29]:
SVD_df.shape, SVD_term_vect.shape

((4100, 50), (1, 50))

In [30]:
SVD_df['cosine_sim'] = cosine_similarity(SVD_df, SVD_term_vect)

### Final Result to Search

In [32]:
SVD_df[['cosine_sim']].sort_values('cosine_sim', ascending=False).head(6)

cosine_sim
mysql                                                 1.000000
cm make simple cmsm free open source gpl conten...    0.829092
exponent cm free open source open standard modu...    0.809465
virtuemart formerly know mambo phpshop open sou...    0.808613
joomla free open source content management syst...    0.806594
pluck conjurer know pluck cm open source conten...    0.799721